# HTTP Client Example

This notebook demonstrates how to use the `HTTPRegistryClient` to connect to a remote LangGate service for model information.

In [1]:
import os
from pprint import pprint as pp
from pydantic import SecretStr

# Configure server URL
LANGGATE_URL = "http://localhost:4000/api/v1"

# If running the langgate server behind Envoy proxy in docker:
# LANGGATE_URL = "http://localhost:10000/api/v1"

# If running and accessing langgate from within a kubernetes cluster:
# LANGGATE_URL = "http://langgate.ns.svc.cluster.local:10000/api/v1"

# Optional: Set API key (if your LangGate service requires authentication)
api_key = SecretStr("your_api_key")

## Initialize the HTTP Client

In [2]:
from langgate.client.http import HTTPRegistryClient

# Initialize the HTTP client
# HTTPRegistryClient is a singleton
client = HTTPRegistryClient(base_url=LANGGATE_URL, api_key=api_key)

2025-06-17 23:20:36 [debug    ] creating_http_registry_client_singleton
2025-06-17 23:20:36 [debug    ] initialized_base_http_registry_client api_key=SecretStr('**********') base_url=http://localhost:4000/api/v1 model_info_cls=<class 'langgate.core.models.LLMInfo'>
2025-06-17 23:20:36 [debug    ] initialized_http_registry_client_singleton


## List Available Models

In [3]:
try:
    # List available models
    models = await client.list_models()
    print(f"Available models: {len(models)}")
    for model in models[:5]:  # Show first 5 models
        print(f"- {model.id}: {model.name}")
except Exception as e:
    print(f"Error connecting to the LangGate service: {str(e)}")
    print("Note: This example requires a running LangGate service.")

2025-06-17 23:20:41 [debug    ] refreshing_model_cache        
2025-06-17 23:20:41 [debug    ] refreshed_model_cache          model_count=58
Available models: 58
- openai/gpt-4.1: GPT-4.1
- openai/gpt-4.1-mini: GPT-4.1 mini
- openai/gpt-4.1-nano: GPT-4.1 nano
- openai/gpt-4o: GPT-4o
- openai/gpt-4o-mini: GPT-4o mini


## Get Detailed Model Information

In [4]:
try:
    # Get detailed information about a specific model
    if models and len(models) > 0:
        model_id = models[0].id
        model_info = await client.get_model_info(model_id)

        print(f"\nModel: {model_info.name}")
        print(f"Provider: {model_info.provider.name}")

        print("\nModel metadata:")
        pp(model_info.model_dump(exclude_none=True))
except Exception as e:
    print(f"Error fetching model information: {str(e)}")
    print("Note: This example requires a running LangGate service.")


Model: GPT-4.1
Provider: OpenAI

Model metadata:
{'capabilities': {'supports_parallel_tool_calls': True,
                  'supports_prompt_caching': True,
                  'supports_response_schema': True,
                  'supports_system_messages': True,
                  'supports_tool_choice': True,
                  'supports_tools': True,
                  'supports_vision': True},
 'context_window': {'max_input_tokens': 1047576, 'max_output_tokens': 32768},
 'costs': {'cache_read_input_token_cost': Decimal('5E-7'),
           'input_cost_per_token': Decimal('0.000002'),
           'output_cost_per_token': Decimal('0.000008')},
 'description': "GPT-4.1 is the latest iteration of OpenAI's flagship model "
                'with improved capabilities across all domains.',
 'id': 'openai/gpt-4.1',
 'name': 'GPT-4.1',
 'provider': {'id': 'openai', 'name': 'OpenAI'},
 'provider_id': 'openai',
 'updated_dt': datetime.datetime(2025, 6, 17, 22, 15, 17, 856172, tzinfo=TzInfo(UTC))}
